In [130]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

offers_recieved = pd.read_csv('for_model.csv')
#offers_recieved = offers_recieved.drop(['id_x', 'id_y','value','person','offer_id','channels','influence_start',
#                                       'influence_end','Unnamed: 0','event','offer_type','index'], axis=1)


transcript = pd.read_json('data/transcript.json', orient='records', lines=True)
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)

#remove top 1% of spending, model will focus too much on these
offers_recieved = offers_recieved[offers_recieved['spending_during_offer'] < offers_recieved['spending_during_offer'].quantile(q=.99)]

offers_recieved.info()
offers_recieved.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17017 entries, 0 to 17188
Data columns (total 22 columns):
Unnamed: 0               17017 non-null int64
offer_type               17017 non-null object
person                   17017 non-null object
time                     17017 non-null int64
difficulty               17017 non-null int64
duration                 17017 non-null int64
reward                   17017 non-null int64
web                      17017 non-null int64
email                    17017 non-null int64
mobile                   17017 non-null int64
social                   17017 non-null int64
bogo                     17017 non-null int64
discount                 17017 non-null int64
informational            17017 non-null int64
age                      17017 non-null int64
income                   17017 non-null float64
member_date_int          17017 non-null int64
F                        17017 non-null int64
M                        17017 non-null int64
typical_spend

,Unnamed: 0,time,difficulty,duration,reward,web,email,mobile,social,bogo,discount,informational,age,income,member_date_int,F,M,typical_spending_m1,num_offers,spending_during_offer
count,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.0,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000,17017.000000
mean,38294.377211,334.227185,7.704413,6.772404,4.525181,0.794441,1.0,0.971382,0.791855,0.422871,0.464418,0.112711,64.182465,65770.022624,1303.206029,0.356232,0.481930,0.164729,2.067462,2.056270
std,21922.154990,195.430578,3.877208,2.060776,3.413907,0.404121,0.0,0.166736,0.405993,0.494030,0.498747,0.316248,27.261797,18510.039129,403.798460,0.478899,0.499688,0.189353,1.032515,5.721521
min,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,30000.000000,1.000000,0.000000,0.000000,0.000071,1.000000,0.000000
25%,19256.000000,168.000000,5.000000,5.000000,2.000000,1.000000,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,46.000000,54000.000000,1025.000000,0.000000,0.000000,0.037141,1.000000,0.000000
50%,38458.000000,408.000000,10.000000,7.000000,3.000000,1.000000,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,59.000000,65405.000000,1459.000000,0.000000,0.000000,0.117070,2.000000,0.000000
75%,57255.000000,504.000000,10.000000,7.000000,5.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,75.000000,76000.000000,1609.000000,1.000000,1.000000,0.232830,3.000000,0.000000
max,76271.000000,576.000000,20.000000,10.000000,10.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,118.000000,120000.000000,1823.000000,1.000000,1.000000,2.308020,6.000000,31.670000


In [132]:
print(offers_recieved.describe())
print(offers_recieved.head())

         Unnamed: 0          time    difficulty      duration        reward  \
count  17017.000000  17017.000000  17017.000000  17017.000000  17017.000000   
mean   38294.377211    334.227185      7.704413      6.772404      4.525181   
std    21922.154990    195.430578      3.877208      2.060776      3.413907   
min        0.000000      0.000000      0.000000      3.000000      0.000000   
25%    19256.000000    168.000000      5.000000      5.000000      2.000000   
50%    38458.000000    408.000000     10.000000      7.000000      3.000000   
75%    57255.000000    504.000000     10.000000      7.000000      5.000000   
max    76271.000000    576.000000     20.000000     10.000000     10.000000   

                web    email        mobile        social          bogo  \
count  17017.000000  17017.0  17017.000000  17017.000000  17017.000000   
mean       0.794441      1.0      0.971382      0.791855      0.422871   
std        0.404121      0.0      0.166736      0.405993      0.49

In [133]:
offers_for_model = offers_recieved.drop(['Unnamed: 0','offer_type','person'],axis=1)

In [134]:
X, y = offers_for_model.iloc[:,:-1],offers_recieved.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=True)

print(X.head())
print(y.head())




    

   time  difficulty  duration  reward  web  email  mobile  social  bogo  \
0     0           5         7       5    1      1       1       0     1   
1     0          20        10       5    1      1       0       0     0   
2     0           5         5       5    1      1       1       1     1   
3     0          10         5      10    1      1       1       1     1   
4     0          10         7      10    0      1       1       1     1   

   discount  informational  age    income  member_date_int  F  M  \
0         0              0   75  100000.0             1380  1  0   
1         1              0  118   65405.0             1467  0  0   
2         0              0   65   53000.0             1656  0  1   
3         0              0   69   57000.0             1112  1  0   
4         0              0   20   30000.0             1078  1  0   

   typical_spending_m1  num_offers  
0             0.227529           2  
1             0.006643           2  
2             0.051747       

In [73]:
rmse = []
for i in range(1,100,1):
    for j in range(475,500,25):
        xg = xgb.XGBRegressor(objective="reg:squarederror",colsample_bytree = 0.3, learning_rate = .01,
                    max_depth = 5, n_estimators = j,alpha = i)
        xg.fit(X_train,y_train)
        test_preds = xg.predict(X_test)
        train_preds = xg.predict(X_train)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
        train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
        rmse.append([i,j,train_rmse,test_rmse])
    
rmse = pd.DataFrame(rmse)
rmse.to_csv('rmse.csv')
    

In [136]:
#proof of concept
xg = xgb.XGBRegressor(objective="reg:squarederror",colsample_bytree = 0.3, learning_rate = .05,
                    max_depth = 5, n_estimators = 475,alpha = 2)
xg.fit(X,y)

preds = xg.predict(X)

rmse = np.sqrt(mean_squared_error(y, preds))
print("rmse: %f" % (rmse))

visual = offers_recieved
visual['preds'] = preds
visual.to_csv('visual.csv')


rmse: 4.999280


# Making Model Actionable

ModuleNotFoundError: No module named 'torch'

In [137]:
print(X.head())

   time  difficulty  duration  reward  web  email  mobile  social  bogo  \
0     0           5         7       5    1      1       1       0     1   
1     0          20        10       5    1      1       0       0     0   
2     0           5         5       5    1      1       1       1     1   
3     0          10         5      10    1      1       1       1     1   
4     0          10         7      10    0      1       1       1     1   

   discount  informational  age    income  member_date_int  F  M  \
0         0              0   75  100000.0             1380  1  0   
1         1              0  118   65405.0             1467  0  0   
2         0              0   65   53000.0             1656  0  1   
3         0              0   69   57000.0             1112  1  0   
4         0              0   20   30000.0             1078  1  0   

   typical_spending_m1  num_offers  
0             0.227529           2  
1             0.006643           2  
2             0.051747       

In [138]:
profile = offers_recieved.groupby('person').mean().drop(['Unnamed: 0','spending_during_offer','preds'],axis=1)

#same transformations used in data cleaning to get the same columns in portfolio of offers
portfolio['web'] = pd.Series([0 for i in range(0,len(portfolio))])
portfolio['email'] = pd.Series([0 for i in range(0,len(portfolio))])
portfolio['mobile'] = pd.Series([0 for i in range(0,len(portfolio))])
portfolio['social'] = pd.Series([0 for i in range(0,len(portfolio))])


for index, i in enumerate(portfolio['channels']):
    for j in i:
        portfolio[j][index] = 1
        
portfolio = pd.concat([portfolio, pd.get_dummies(portfolio['offer_type'])], axis=1)
print(portfolio)


      

C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


                       channels  difficulty  duration  \
0       [email, mobile, social]          10         7   
1  [web, email, mobile, social]          10         5   
2          [web, email, mobile]           0         4   
3          [web, email, mobile]           5         7   
4                  [web, email]          20        10   
5  [web, email, mobile, social]           7         7   
6  [web, email, mobile, social]          10        10   
7       [email, mobile, social]           0         3   
8  [web, email, mobile, social]           5         5   
9          [web, email, mobile]          10         7   

                                 id     offer_type  reward  web  email  \
0  ae264e3637204a6fb9bb56bc8210ddfd           bogo      10    0      1   
1  4d5c57ea9a6940dd891ad53e9dbe8da0           bogo      10    1      1   
2  3f207df678b143eea3cee63160fa8bed  informational       0    1      1   
3  9b98b8c7a33c4b65b9aebfe6a799e6d9           bogo       5    1      1   
4 

C:\Users\duran\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value i

1
0
1
0
0
0
0
1
0


ValueError: Shape of passed values is (1, 10), indices imply (19, 10)

In [143]:
def predict_portfolio(profile,portfolio,person,time):
    person_port_o = profile[profile.index == person]
    l = []
    for i in range(0,len(portfolio)):
        if i == 0:
            person_port = person_port_o
            person_port.loc[person,'time'] = time
            person_port.loc[person,'difficulty'] = portfolio.loc[i,'difficulty']
            person_port.loc[person,'duration'] = portfolio.loc[i,'duration']
            person_port.loc[person,'reward'] = portfolio.loc[i,'reward']
            person_port.loc[person,'web'] = portfolio.loc[i,'web']
            person_port.loc[person,'email'] = portfolio.loc[i,'email']
            person_port.loc[person,'mobile'] = portfolio.loc[i,'mobile']
            person_port.loc[person,'social'] = portfolio.loc[i,'social']
            person_port.loc[person,'bogo'] = portfolio.loc[i,'bogo']
            person_port.loc[person,'discount'] = portfolio.loc[i,'discount']
            person_port.loc[person,'informational'] = portfolio.loc[i,'informational']
            person_port['offer'] = i
        else:
            person_port_o
            person_port_o.loc[person,'time'] = time
            person_port_o.loc[person,'difficulty'] = portfolio.loc[i,'difficulty']
            person_port_o.loc[person,'duration'] = portfolio.loc[i,'duration']
            person_port_o.loc[person,'reward'] = portfolio.loc[i,'reward']
            person_port_o.loc[person,'web'] = portfolio.loc[i,'web']
            person_port_o.loc[person,'email'] = portfolio.loc[i,'email']
            person_port_o.loc[person,'mobile'] = portfolio.loc[i,'mobile']
            person_port_o.loc[person,'social'] = portfolio.loc[i,'social']
            person_port_o.loc[person,'bogo'] = portfolio.loc[i,'bogo']
            person_port_o.loc[person,'discount'] = portfolio.loc[i,'discount']
            person_port_o.loc[person,'informational'] = portfolio.loc[i,'informational']
            person_port_o['offer'] = i
            person_port.append(person_port_o,ignore_index=True)
        
    return pd.DataFrame(person_port)
        
        
    


#print(profile.iloc[0])
print(predict_portfolio(profile,portfolio,'0020ccbbb6d84e358d3414a3ff76cffd',0))
    

C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is tryi

                                  time  difficulty  duration  reward  web  \
person                                                                      
0020ccbbb6d84e358d3414a3ff76cffd   0.0        10.0       7.0     2.0  1.0   

                                  email  mobile  social  bogo  discount  \
person                                                                    
0020ccbbb6d84e358d3414a3ff76cffd    1.0     1.0     0.0   0.0       1.0   

                                  informational   age   income  \
person                                                           
0020ccbbb6d84e358d3414a3ff76cffd            0.0  24.0  60000.0   

                                  member_date_int    F    M  \
person                                                        
0020ccbbb6d84e358d3414a3ff76cffd           1201.0  1.0  0.0   

                                  typical_spending_m1  num_offers  offer  
person                                                                    
002